In [53]:
!python --version

Python 3.7.13


In [54]:
!pip install git+https://github.com/carla-recourse/carla.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/carla-recourse/carla.git to /tmp/pip-req-build-_x56h6sd
  Running command git clone -q https://github.com/carla-recourse/carla.git /tmp/pip-req-build-_x56h6sd


# Load Adult dataset

In [55]:
from carla.data.catalog import OnlineCatalog

# load catalog dataset
data_name = "adult"
dataset = OnlineCatalog(data_name)

# Train ML model

In [56]:
!rm /root/carla/models/adult/ann_layers_18_9_3.pt

In [57]:
from carla.models.catalog import MLModelCatalog

training_params = {"lr": 0.002, "epochs": 10, "batch_size": 1024, "hidden_size": [18, 9, 3]}

ml_model = MLModelCatalog(
    dataset,
    model_type="ann",
    load_online=False,
    cache=False,
    backend="pytorch"
)

ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"]
)

balance on test set 0.23883245958934032, balance on test set 0.2408256880733945
Epoch 0/9
----------


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.4800 Acc: 0.7812

test Loss: 0.4130 Acc: 0.8127

Epoch 1/9
----------
train Loss: 0.4001 Acc: 0.8044

test Loss: 0.3899 Acc: 0.8122

Epoch 2/9
----------
train Loss: 0.3867 Acc: 0.8146

test Loss: 0.3868 Acc: 0.8233

Epoch 3/9
----------
train Loss: 0.3773 Acc: 0.8216

test Loss: 0.3794 Acc: 0.8254

Epoch 4/9
----------
train Loss: 0.3687 Acc: 0.8288

test Loss: 0.3683 Acc: 0.8301

Epoch 5/9
----------
train Loss: 0.3631 Acc: 0.8299

test Loss: 0.3577 Acc: 0.8367

Epoch 6/9
----------
train Loss: 0.3564 Acc: 0.8335

test Loss: 0.3523 Acc: 0.8383

Epoch 7/9
----------
train Loss: 0.3525 Acc: 0.8342

test Loss: 0.3557 Acc: 0.8348

Epoch 8/9
----------
train Loss: 0.3492 Acc: 0.8362

test Loss: 0.3451 Acc: 0.8405

Epoch 9/9
----------
train Loss: 0.3436 Acc: 0.8384

test Loss: 0.3530 Acc: 0.8344



# Get negatively labeled samples for which we would like to find counterfactuals

In [58]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

display(test_factual)

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.02174,0.0,...,0.0,1.0,1.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
6,0.438356,0.100061,0.266667,0.00000,0.0,...,1.0,0.0,1.0,0.0,1.0
12,0.082192,0.074410,0.800000,0.00000,0.0,...,0.0,1.0,1.0,0.0,1.0


In [59]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}
recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
0,0.350678,0.089731,0.849348,0.071152,0.049423,...,0.0,1.0,1.0,1.0,1.0
2,0.414628,0.258440,0.660331,0.127013,0.126985,...,1.0,1.0,1.0,1.0,1.0
3,0.542452,0.192815,0.449352,0.049444,0.049468,...,1.0,0.0,0.0,1.0,1.0
6,0.636830,0.078099,0.466372,0.200574,0.200450,...,1.0,1.0,0.0,0.0,0.0
12,0.170705,0.159599,0.888211,0.088278,0.088297,...,0.0,1.0,1.0,0.0,1.0


Select all white people and change their race to non-white to see the difference in prediction

In [60]:
import numpy as np
white_df = dataset.df[dataset.df['race_White']==1]
white_df
pred = ml_model.predict(white_df)
print(np.mean(pred))
white_changed_df = white_df
white_changed_df['race_White']=0
pred2 = ml_model.predict(white_changed_df)
print(np.mean(pred2))

0.3076805
0.29806003


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


Same strategy with male and female profiles

In [61]:
male_df = dataset.df[dataset.df['sex_Male']==1]
pred = ml_model.predict(male_df)
print(np.mean(pred))

male_changed_df = male_df
male_changed_df['sex_Male']=0

pred2 = ml_model.predict(male_changed_df)
print(np.mean(pred2))

0.36171794
0.34717116


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


And with age

In [62]:
import numpy as np
age_df = dataset.df
age_df['age']=0.5
pred = ml_model.predict(age_df)
print(np.mean(pred))

age_changed_df = age_df
age_changed_df['age']=0.4

pred2 = ml_model.predict(age_changed_df)
print(np.mean(pred2))

0.33155432
0.30885327


Calculating confusion matrix for minorities

In [70]:
white_df = dataset.df[dataset.df['sex_Male']==0]
white_df_y = dataset.df[dataset.df['sex_Male']==0]['income']

pred = ml_model.predict(white_df)

/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [71]:
from sklearn.metrics import confusion_matrix

In [72]:
tn, fp, fn, tp = confusion_matrix(white_df_y,np.round(pred)).ravel()

In [73]:
print('tn',tn,'fp',fp,'fn',fn,'tp',tp)

tn 13947 fp 473 fn 818 tp 951


In [74]:
precision = tp/(tp+fp)
print(precision)
recall = tp/(tp+fn)
print(recall)

0.6678370786516854
0.537591859807801


In [75]:
white_df = dataset.df[dataset.df['sex_Male']==1]
white_df_y = dataset.df[dataset.df['sex_Male']==1]['income']

pred = ml_model.predict(white_df)
tn, fp, fn, tp = confusion_matrix(white_df_y,np.round(pred)).ravel()
print('tn',tn,'fp',fp,'fn',fn,'tp',tp)
precision = tp/(tp+fp)
print(precision)
recall = tp/(tp+fn)
print(recall)

tn 19124 fp 3601 fn 3191 tp 6727
0.6513361735089078
0.6782617463198225


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


Compute distance between observations and their corresponding counterfactuals

In [87]:
female_df = dataset.df[dataset.df['sex_Male']==0]
male_df = dataset.df[dataset.df['sex_Male']==1]

hyperparams = {"loss_type": "BCE", "binary_cat_features": True}
recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)

factuals_female = predict_negative_instances(ml_model, female_df)
test_factual_female = factuals_female.iloc[:10]
df_cfs_female = recourse_method.get_counterfactuals(test_factual_female)

display(test_factual_female)
display(df_cfs_female)

from sklearn.metrics import pairwise_distances
dist = pairwise_distances(test_factual_female[df_cfs_female.columns],df_cfs_female)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
6,0.438356,0.100061,0.266667,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0
12,0.082192,0.074410,0.800000,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0
19,0.356164,0.189356,0.866667,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0
21,0.506849,0.196102,0.533333,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0
24,0.575342,0.065441,0.533333,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0
37,0.027397,0.359787,0.533333,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0
43,0.438356,0.055715,0.533333,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0
47,0.369863,0.078525,0.866667,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0
50,0.109589,0.013524,0.600000,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0
51,0.013699,0.145233,0.533333,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
6,0.636830,0.078099,0.466372,0.200574,0.200450,...,1.0,1.0,0.0,0.0,0.0
12,0.170705,0.159599,0.888211,0.088278,0.088297,...,0.0,1.0,1.0,0.0,1.0
19,0.395996,0.227795,0.906214,0.039611,0.039642,...,0.0,1.0,1.0,0.0,1.0
21,0.652070,0.336175,0.678842,0.145476,0.145391,...,1.0,1.0,0.0,0.0,1.0
24,0.683293,0.168317,0.641316,0.107995,0.107973,...,1.0,1.0,1.0,0.0,1.0
37,0.076836,0.405617,0.582790,0.049464,0.049451,...,0.0,1.0,1.0,0.0,1.0
43,0.536545,0.149001,0.631552,0.098231,0.098209,...,0.0,1.0,1.0,0.0,1.0
47,0.419328,0.124558,0.916149,0.049489,0.049477,...,0.0,1.0,1.0,0.0,1.0
50,0.129542,0.033432,0.619958,0.019959,0.019956,...,0.0,1.0,0.0,0.0,1.0
51,0.170060,0.295141,0.689740,0.156424,0.156392,...,1.0,1.0,1.0,0.0,1.0


In [96]:
np.mean(dist.diagonal())

0.22839883482108553

In [97]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}
recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)

factuals_male = predict_negative_instances(ml_model, male_df)
test_factual_male = factuals_male.iloc[:10]
df_cfs_male = recourse_method.get_counterfactuals(test_factual_male)

display(test_factual_male)
display(df_cfs_male)

dist = pairwise_distances(test_factual_male[df_cfs_female.columns],df_cfs_male)
np.mean(dist.diagonal())

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.02174,0.0,...,0.0,1.0,1.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
13,0.205479,0.130392,0.733333,0.00000,0.0,...,1.0,0.0,1.0,1.0,1.0
14,0.315068,0.074072,0.666667,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
15,0.232877,0.157770,0.200000,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
16,0.109589,0.111271,0.533333,0.00000,0.0,...,1.0,1.0,1.0,1.0,0.0
17,0.205479,0.118082,0.533333,0.00000,0.0,...,0.0,1.0,1.0,1.0,1.0
18,0.287671,0.011232,0.400000,0.00000,0.0,...,1.0,1.0,0.0,1.0,1.0
22,0.246575,0.043677,0.266667,0.00000,0.0,...,1.0,0.0,0.0,1.0,0.0


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
0,0.350678,0.089731,0.849348,0.071152,0.049423,...,0.0,1.0,1.0,1.0,1.0
2,0.414628,0.258440,0.660331,0.127013,0.126985,...,1.0,1.0,1.0,1.0,1.0
3,0.542452,0.192815,0.449352,0.049444,0.049468,...,1.0,0.0,0.0,1.0,1.0
13,0.293839,0.213216,0.821735,0.088410,0.088387,...,1.0,1.0,0.0,1.0,1.0
14,0.325068,0.084072,0.676667,0.010000,0.010000,...,1.0,0.0,0.0,1.0,1.0
15,0.330713,0.254501,0.297868,0.097881,0.097857,...,1.0,0.0,0.0,1.0,0.0
16,0.246260,0.241647,0.670098,0.136931,0.136970,...,1.0,1.0,1.0,1.0,1.0
17,0.312838,0.204103,0.640833,0.107796,0.107893,...,0.0,1.0,1.0,1.0,1.0
18,0.337154,0.060133,0.449498,0.049505,0.049493,...,1.0,0.0,1.0,1.0,1.0
22,0.334877,0.131001,0.354996,0.088341,0.088320,...,1.0,0.0,0.0,1.0,1.0


0.19495886045835764

In [99]:
minor_df = dataset.df[dataset.df['race_White']==0]
major_df = dataset.df[dataset.df['race_White']==1]

hyperparams = {"loss_type": "BCE", "binary_cat_features": True}
recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)

factuals_minor = predict_negative_instances(ml_model, minor_df)
test_factual_minor = factuals_minor.iloc[:10]
df_cfs_minor = recourse_method.get_counterfactuals(test_factual_minor)

display(test_factual_minor)
display(df_cfs_minor)

dist = pairwise_distances(test_factual_minor[df_cfs_minor.columns],df_cfs_minor)
np.mean(dist.diagonal())

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
3,0.493151,0.150486,0.400000,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0
6,0.438356,0.100061,0.266667,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0
13,0.205479,0.130392,0.733333,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0
14,0.315068,0.074072,0.666667,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0
15,0.232877,0.157770,0.200000,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0
21,0.506849,0.196102,0.533333,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0
22,0.246575,0.043677,0.266667,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0
31,0.041096,0.171658,0.600000,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0
34,0.068493,0.202438,0.600000,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0
50,0.109589,0.013524,0.600000,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
3,0.542452,0.192815,0.449352,0.049444,0.049468,...,1.0,0.0,0.0,1.0,1.0
6,0.636830,0.078099,0.466372,0.200574,0.200450,...,1.0,1.0,0.0,0.0,0.0
13,0.293839,0.213216,0.821735,0.088410,0.088387,...,1.0,1.0,0.0,1.0,1.0
14,0.325068,0.084072,0.676667,0.010000,0.010000,...,1.0,0.0,0.0,1.0,1.0
15,0.330713,0.254501,0.297868,0.097881,0.097857,...,1.0,0.0,0.0,1.0,0.0
21,0.652070,0.336175,0.678842,0.145476,0.145391,...,1.0,1.0,0.0,0.0,1.0
22,0.334877,0.131001,0.354996,0.088341,0.088320,...,1.0,0.0,0.0,1.0,1.0
31,0.168356,0.293204,0.727298,0.127313,0.127286,...,1.0,1.0,0.0,1.0,1.0
34,0.137338,0.270423,0.668869,0.068878,0.068860,...,1.0,0.0,0.0,1.0,1.0
50,0.129542,0.033432,0.619958,0.019959,0.019956,...,0.0,1.0,0.0,0.0,1.0


0.2141384813152866

In [100]:
factuals_major = predict_negative_instances(ml_model, major_df)
test_factual_major = factuals_major.iloc[:10]
df_cfs_major = recourse_method.get_counterfactuals(test_factual_major)

dist = pairwise_distances(test_factual_major[df_cfs_major.columns],df_cfs_major)
np.mean(dist.diagonal())

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/usr/local/lib/python3.7/dist-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


0.20694417450237795

Use the benchmark module to evaluate counterfactuals and get redundancy

In [124]:
minor_df = dataset.df[dataset.df['sex_Male']==0]
major_df = dataset.df[dataset.df['sex_Male']==1]
factuals_minor = predict_negative_instances(ml_model, minor_df)
factuals_major = predict_negative_instances(ml_model, major_df)

test_factual_minor = factuals_minor.iloc[:10]
test_factual_major = factuals_major.iloc[:10]


In [127]:
from carla import Benchmark
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
benchmark = Benchmark(ml_model, recourse_method, test_factual_minor)

# now you can decide if you want to run all measurements
# or just specific ones.
evaluation_measures = [
    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
    evaluation_catalog.Distance(benchmark.mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
    evaluation_catalog.AvgTime({"time": benchmark.timer}),
]

# now run all implemented measurements and create a
# DataFrame which consists of all results
results = benchmark.run_benchmark(evaluation_measures)

display(results.head(5))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


,y-Nearest-Neighbours,L0_distance,L1_distance,L2_distance,Linf_distance,Success_Rate,Redundancy,Constraint_Violation,avg_time
0,0.04,6.0,1.022118,0.200549,0.200953,1.0,1,1,0.029885
1,NaN,6.0,0.526707,0.046245,0.088513,NaN,1,1,NaN
2,NaN,6.0,0.236636,0.009334,0.039832,NaN,1,1,NaN
3,NaN,6.0,0.868462,0.125732,0.146793,NaN,1,1,NaN
4,NaN,6.0,0.642745,0.068875,0.107995,NaN,1,1,NaN


In [128]:
display(results)

,y-Nearest-Neighbours,L0_distance,L1_distance,L2_distance,Linf_distance,Success_Rate,Redundancy,Constraint_Violation,avg_time
0,0.04,6.0,1.022118,0.200549,0.200953,1.0,1,1,0.029885
1,NaN,6.0,0.526707,0.046245,0.088513,NaN,1,1,NaN
2,NaN,6.0,0.236636,0.009334,0.039832,NaN,1,1,NaN
3,NaN,6.0,0.868462,0.125732,0.146793,NaN,1,1,NaN
4,NaN,6.0,0.642745,0.068875,0.107995,NaN,1,1,NaN
5,NaN,6.0,0.293091,0.014328,0.049464,NaN,4,1,NaN
6,NaN,6.0,0.584339,0.056929,0.098231,NaN,1,1,NaN
7,NaN,6.0,0.293420,0.014359,0.049489,NaN,3,1,NaN
8,NaN,6.0,0.119690,0.002388,0.019959,NaN,2,1,NaN
9,NaN,6.0,0.931879,0.144768,0.156424,NaN,1,1,NaN


In [129]:
np.mean(results['Redundancy'])

1.6